In [1]:
import pandas as pd
import plotly.graph_objects as go

# Wczytanie danych z pliku CSV
data = pd.read_csv('men_10000m.csv')

# Obliczanie liczby wystąpień każdego kraju
country_counts = data['Country'].value_counts()

# Wybieranie top 10 krajów
top_10_countries = country_counts.head(10)

# Tworzenie wykresu słupkowego
fig = go.Figure(data=[
    go.Bar(x=top_10_countries.index, y=top_10_countries.values)
])

# Dodawanie liczby wystąpień nad słupkami
fig.update_traces(text=top_10_countries.values, textposition='outside')

# Aktualizowanie tytułu i osi
fig.update_layout(
    title='Top 10 - liczba najlepszych wyników na 1000m według kraju',
    xaxis_title='Kraj',
    yaxis_title='Liczba wyników',
    height= 700
)

# Zapisywanie wykresu do pliku HTML
fig.write_html('top_10_countries.html')

In [3]:
import pandas as pd
import plotly.express as px
import re

# Funkcja do przeliczania czasu w formacie mm:ss.s na wartość liczbową w sekundach
def convert_time_to_seconds(time_string):
    time_parts = time_string.split(':')
    minutes = int(time_parts[0])
    seconds = float(time_parts[1])
    total_seconds = minutes * 60 + seconds
    return total_seconds

# Funkcja do usunięcia litery na końcu wartości
def remove_last_character(value):
    pattern = r'[^0-9.:]'  # Wzorzec do dopasowania nieliczbowych, nieliterowych i niekropkowych znaków
    return re.sub(pattern, '', value)

# Wczytanie danych z pliku CSV
data = pd.read_csv('men_10000m.csv')

# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na wartość liczbową w sekundach
data['Time'] = data['Time'].apply(convert_time_to_seconds)

# Sortowanie wyników względem daty i czasu
data = data.sort_values(by=['Event Date', 'Time'])

# Inicjalizacja pustej listy do przechowywania najlepszych wyników
best_results = []

# Ustalenie daty początkowej
start_date = pd.to_datetime('1968-10-18', format='%Y-%m-%d')  # Zmiana daty początkowej

# Przeszukiwanie posortowanych danych w celu znalezienia progresji rekordu świata
current_best_time = float('inf')
for _, row in data.iterrows():
    if row['Event Date'] >= start_date:
        if row['Time'] < current_best_time:
            best_results.append(row)
            current_best_time = row['Time']

# Konwersja listy wyników do DataFrame
best_results = pd.DataFrame(best_results)

# Tworzenie wykresu punktowego
fig = px.scatter(best_results, x='Event Date', y='Time', title='Progresja rekordu świata - 10000m mężczyzn')
fig.update_traces(mode='markers+lines')  # Dodanie linii łączących punkty

# Aktualizacja parametrów osi y
fig.update_layout(
    yaxis=dict(
        title='Najlepszy czas',
        tickvals=[1710, 1680, 1650, 1620,1590, 1560],  # Wartości osi Y w sekundach
        ticktext=['28:30.0','28:00.0','27:30.0', '27:00.0','26:30.0', '26:00.0'],  # Etykiety osi Y
        tickformat='.0f'  # Formatowanie etykiet jako tekst
    ),
    xaxis_title='Data biegu',
    height = 700,
)

# Zapisywanie wykresu do pliku HTML
fig.write_html("WR_progression.html")
